Problem 1
● Current Stock Price $151.03
● Strike Price $165
● Current Date 03/13/2022
● Options Expiration Date 04/15/2022
● Risk Free Rate of 4.25%
● Continuously Compounding Coupon of 0.53%
Implement the closed form greeks for GBSM. Implement a finite difference derivative calculation.
Compare the values between the two methods for both a call and a put.
Implement the binomial tree valuation for American options with and without discrete dividends. Assume
the stock above:
● Pays dividend on 4/11/2022 of $0.88
Calculate the value of the call and the put. Calculate the Greeks of each.
What is the sensitivity of the put and call to a change in the dividend amount?
实现GBSM（Black-Scholes-Merton模型）的闭式Greeks计算。实现有限差分法的导数计算。比较两种方法在看涨期权和看跌期权上的计算值。

实现美式期权的二叉树估值，分别考虑有分红和无分红的情况。假设上述股票：

在2022年4月11日支付分红$0.88。
计算看涨和看跌期权的价值，以及每个期权的Greeks。

分析看涨和看跌期权对分红金额变化的敏感性。


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [7]:
def binomial_tree_american_option(S, K, T, r, q, sigma, steps, option_type="call", dividend_payment=None, dividend_date=None):
    """
    使用二叉树估值美式期权价格（包括分红调整）
    参数：
        S: 当前股价
        K: 行权价
        T: 到期时间（年）
        r: 无风险利率
        q: 连续复利股息率
        sigma: 波动率
        steps: 二叉树步数
        option_type: "call" 或 "put" 表示看涨或看跌期权
        dividend_payment: 分红金额（如果有）
        dividend_date: 分红日期（以年为单位），即从当前日期开始的时间
    返回：
        期权价格
    """
    dt = T / steps  # 每步的时间间隔
    u = np.exp(sigma * np.sqrt(dt))  # 上升比例
    d = 1 / u  # 下降比例
    p = (np.exp((r - q) * dt) - d) / (u - d)  # 风险中性概率

    # 初始化股票价格树
    stock_tree = np.zeros((steps + 1, steps + 1))
    for i in range(steps + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S * (u ** (i - j)) * (d ** j)
            # 如果设置了分红，在分红日期调整股票价格
            if dividend_payment is not None and dividend_date is not None:
                if i * dt >= dividend_date:
                    stock_tree[j, i] -= dividend_payment

    # 初始化期权价值树
    option_tree = np.zeros((steps + 1, steps + 1))
    for j in range(steps + 1):
        if option_type == "call":
            option_tree[j, steps] = max(0, stock_tree[j, steps] - K)  # 看涨期权的到期价值
        elif option_type == "put":
            option_tree[j, steps] = max(0, K - stock_tree[j, steps])  # 看跌期权的到期价值

    # 反向迭代计算期权价值
    for i in range(steps - 1, -1, -1):
        for j in range(i + 1):
            # 折现后的期权持有价值
            option_value_if_held = np.exp(-r * dt) * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])
            # 美式期权可以提前行权，取持有价值和立即行权价值的较大值
            if option_type == "call":
                option_tree[j, i] = max(option_value_if_held, stock_tree[j, i] - K)
            elif option_type == "put":
                option_tree[j, i] = max(option_value_if_held, K - stock_tree[j, i])

    return option_tree[0, 0]

# 参数设置
S = 151.03  # 当前股价
K = 165  # 行权价
T = (30 + 15 + 1) / 365  # 从3月13日到4月15日的到期时间（年）
r = 0.0425  # 无风险利率
q = 0.0053  # 年化股息率（假设为连续复利）
sigma = 0.20  # 隐含波动率
steps = 1000  # 二叉树步数，越高精度越高

# 计算无分红情况下的美式看涨和看跌期权价格
call_price_no_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "call")
put_price_no_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "put")

# 计算有分红情况下的美式看涨和看跌期权价格
dividend_payment = 0.88  # 每股分红
dividend_date = (30 + 11) / 365  # 分红日期为4月11日，折算成年
call_price_with_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "call", dividend_payment, dividend_date)
put_price_with_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "put", dividend_payment, dividend_date)

# 输出结果
print(f"无分红情况下的美式看涨期权价格: {call_price_no_dividend:.2f}")
print(f"无分红情况下的美式看跌期权价格: {put_price_no_dividend:.2f}")
print(f"有分红情况下的美式看涨期权价格: {call_price_with_dividend:.2f}")
print(f"有分红情况下的美式看跌期权价格: {put_price_with_dividend:.2f}")

无分红情况下的美式看涨期权价格: 0.65
无分红情况下的美式看跌期权价格: 14.18
有分红情况下的美式看涨期权价格: 0.60
有分红情况下的美式看跌期权价格: 14.69


In [8]:
# 计算Greeks
def calculate_greeks(S, K, T, r, q, sigma):
    """
    计算Black-Scholes模型的Greeks：Delta, Gamma, Vega, Theta, Rho
    """
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Delta
    delta_call = np.exp(-q * T) * norm.cdf(d1)
    delta_put = np.exp(-q * T) * (norm.cdf(d1) - 1)
    
    # Gamma
    gamma = (np.exp(-q * T) * norm.pdf(d1)) / (S * sigma * np.sqrt(T))
    
    # Vega
    vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
    
    # Theta
    theta_call = (-S * norm.pdf(d1) * sigma * np.exp(-q * T) / (2 * np.sqrt(T)) -
                  r * K * np.exp(-r * T) * norm.cdf(d2) +
                  q * S * np.exp(-q * T) * norm.cdf(d1))
    
    theta_put = (-S * norm.pdf(d1) * sigma * np.exp(-q * T) / (2 * np.sqrt(T)) +
                 r * K * np.exp(-r * T) * norm.cdf(-d2) -
                 q * S * np.exp(-q * T) * norm.cdf(-d1))
    
    # Rho
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2)
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return {
        "Delta": {"call": delta_call, "put": delta_put},
        "Gamma": gamma,
        "Vega": vega,
        "Theta": {"call": theta_call, "put": theta_put},
        "Rho": {"call": rho_call, "put": rho_put}
    }

# 计算并显示Greeks
greeks = calculate_greeks(S, K, T, r, q, sigma)
print("Greeks计算结果：")
print(f"Delta (看涨): {greeks['Delta']['call']:.4f}")
print(f"Delta (看跌): {greeks['Delta']['put']:.4f}")
print(f"Gamma: {greeks['Gamma']:.4f}")
print(f"Vega: {greeks['Vega']:.4f}")
print(f"Theta (看涨): {greeks['Theta']['call']:.4f}")
print(f"Theta (看跌): {greeks['Theta']['put']:.4f}")
print(f"Rho (看涨): {greeks['Rho']['call']:.4f}")
print(f"Rho (看跌): {greeks['Rho']['put']:.4f}")


Greeks计算结果：
Delta (看涨): 0.1261
Delta (看跌): -0.8732
Gamma: 0.0193
Vega: 11.1044
Theta (看涨): -9.4920
Theta (看跌): -3.3168
Rho (看涨): 2.3185
Rho (看跌): -18.3650


In [9]:
# 计算有限差分法Greeks
def finite_difference_greeks(S, K, T, r, q, sigma, epsilon=1e-4):
    """
    使用有限差分法近似Greeks：Delta, Gamma, Vega
    """
    # Delta
    price_up = black_scholes_call_put(S + epsilon, K, T, r, q, sigma, "call")
    price_down = black_scholes_call_put(S - epsilon, K, T, r, q, sigma, "call")
    delta_fd = (price_up - price_down) / (2 * epsilon)
    
    # Gamma
    gamma_fd = (price_up - 2 * black_scholes_call_put(S, K, T, r, q, sigma, "call") + price_down) / (epsilon ** 2)
    
    # Vega
    price_vega = black_scholes_call_put(S, K, T, r, q, sigma + epsilon, "call")
    vega_fd = (price_vega - call_price) / epsilon
    
    return {"Delta (FD)": delta_fd, "Gamma (FD)": gamma_fd, "Vega (FD)": vega_fd}

# 显示有限差分法Greeks
fd_greeks = finite_difference_greeks(S, K, T, r, q, sigma)
print("有限差分法Greeks计算结果：")
print(f"Delta (FD): {fd_greeks['Delta (FD)']:.4f}")
print(f"Gamma (FD): {fd_greeks['Gamma (FD)']:.4f}")
print(f"Vega (FD): {fd_greeks['Vega (FD)']:.4f}")

有限差分法Greeks计算结果：
Delta (FD): 0.1261
Gamma (FD): 0.0193
Vega (FD): 431.7225


Problem 2
Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :
● American Options
● Current Date 03/03/2023
● Current AAPL price is 165
● Risk Free Rate of 4.25%
● Dividend Payment of $1.00 on 3/15/2023
Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). Calculate Mean, VaR and ES.
Calculate VaR and ES using Delta-Normal.
Present all VaR and ES values a $ loss, not percentages.
Compare these results to last week’s results.

使用DailyPrices.csv，对AAPL的收益率拟合正态分布，并假设收益率均值为0。模拟未来10天的AAPL收益，并将这些收益应用于当前AAPL股价（即165美元）。计算以下指标：
均值、VaR（风险价值）和ES（预期损失）
使用Delta-Normal方法计算VaR和ES
所有VaR和ES结果应以美元损失的方式表示，而非百分比。将这些结果与上周的结果进行比较。

In [15]:
from scipy.stats import norm
from scipy.optimize import brentq
import numpy as np

def black_scholes_price(S, X, ttm, rf, q, sigma, option_type="call"):
    """Black-Scholes期权定价公式."""
    d1 = (np.log(S / X) + (rf - q + 0.5 * sigma ** 2) * ttm) / (sigma * np.sqrt(ttm))
    d2 = d1 - sigma * np.sqrt(ttm)
    
    if option_type == "call":
        price = S * np.exp(-q * ttm) * norm.cdf(d1) - X * np.exp(-rf * ttm) * norm.cdf(d2)
    else:
        price = X * np.exp(-rf * ttm) * norm.cdf(-d2) - S * np.exp(-q * ttm) * norm.cdf(-d1)
    
    return price

def implied_volatility(S, X, ttm, rf, price):
    """求解隐含波动率."""
    def objective(sigma):
        return black_scholes_price(S, X, ttm, rf, rf, sigma) - price
    
    iv = brentq(objective, 0.01, 2.0)  # 使用Brent方法在0.01到2.0之间求解
    return iv

# 输入参数
S = 100.50
X = 100
ttm = 15 / 255
rf = 0.0025
p = 2.5

# 计算隐含波动率
implied_vol = implied_volatility(S, X, ttm, rf, p)
print(f"隐含波动率: {implied_vol:.4f}")


隐含波动率: 0.2311


In [16]:
nSim = 5000  # 模拟次数
days_ahead = 10
daily_volatility = implied_vol / np.sqrt(255)
returns = np.random.normal(0, daily_volatility, (nSim, days_ahead))
price_paths = S * np.exp(np.cumsum(returns, axis=1))  # 累积收益转化为价格路径

In [17]:
# 到期时间减1天
ttm_new = 14 / 255

# 计算未对冲的损益
initial_option_value = p
pnl = -100 * (np.array([black_scholes_price(S_new, X, ttm_new, rf, rf, implied_vol)
                        for S_new in price_paths[:, -1]]) - initial_option_value)

# 计算VaR和ES
var_unhedged = np.percentile(pnl, 5)  # 95% VaR
es_unhedged = pnl[pnl <= var_unhedged].mean()  # 95% ES

print(f"VaR 未对冲: {var_unhedged:.2f}")
print(f"ES 未对冲 : {es_unhedged:.2f}")


VaR 未对冲: -596.95
ES 未对冲 : -816.41


In [18]:
# 计算期权的Delta
d1 = (np.log(S / X) + (rf + 0.5 * implied_vol ** 2) * ttm) / (implied_vol * np.sqrt(ttm))
delta = -100 * norm.cdf(d1)

# 更新对冲后的PnL，考虑股票头寸的变化
pnl_hedged = pnl + delta * (price_paths[:, -1] - S)

# 重新计算VaR和ES
var_hedged = np.percentile(pnl_hedged, 5)  # 95% VaR
es_hedged = pnl_hedged[pnl_hedged <= var_hedged].mean()  # 95% ES

print(f"VaR 对冲: {var_hedged:.2f}")
print(f"ES 对冲 : {es_hedged:.2f}")


VaR 对冲: -1023.78
ES 对冲 : -1368.22
